In [ ]:
import cv2
from sklearn.externals import joblib
from skimage.feature import hog
import numpy as np

# Load the classifier
clf = joblib.load("digits_cls.pkl")

# Read the input image
im = cv2.imread("photo8.jpg")

# Convert to grayscale and apply Gaussian filtering
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)

# Threshold the image
ret, im_th = cv2.threshold(im_gray,90,255,cv2.THRESH_BINARY_INV)

# Find contours in the image
ctrs, hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Get rectangles contains each contour
rects = [cv2.boundingRect(ctr) for ctr in ctrs]

'''
For each rectangular region, calculate HOG features and predict
the digit using Linear SVM.
'''

for rect in rects:
    # Draw the rectangles
    cv2.rectangle(im, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 3)

     # Make the rectangular region around the digit
    leng = int(rect[3] * 1.6)
    pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
    pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
    if pt1<0:
        pt1=0
    if pt2<0:
        pt2=0
    roi = im_th[pt1:pt1+leng, pt2:pt2+leng]

    # Resize the image
    roi = cv2.resize(roi, (28, 28), interpolation=cv2.INTER_AREA)
    roi = cv2.dilate(roi, (3, 3))

    # Calculate the HOG features
    roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False,block_norm="L1")

    #actual prediction
    nbr = clf.predict(np.array([roi_hog_fd], 'float64'))
    cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]),cv2.FONT_HERSHEY_DUPLEX, 2, (0, 255, 255), 3)
cv2.imshow("Resulting Image with Rectangular ROIs", im)
cv2.waitKey(0)


In [ ]:
# Import the modules
from sklearn.externals import joblib
from sklearn import datasets
from skimage.feature import hog
from sklearn.svm import LinearSVC
import numpy as np


In [ ]:

X,y= datasets.fetch_openml('mnist_784', version=1, return_X_y=True)                                                                        #importing datset and labels straight from sklearn
Images = np.array(X, 'int16')                                                                                                              #converting list of Images into a numpy array.
labels = np.array(y, 'int')
list_hog_fd = []
for Image in Images:
    fd = hog(Image.reshape((28, 28)), orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualize=False,block_norm='L2')    # using hog algorithm to extract features from each image
    list_hog_fd.append(fd)                                                                                                                  #adding the hog features extracted from the image into a list
hog_features = np.array(list_hog_fd, 'float64')                                                                                             #converting the HOG feature list into a numpy array

'''
Two line code to train entire dataset
load the LinearSVC() model into clf
and then fit hog_features and list into clf
'''
clf = LinearSVC()
clf.fit(hog_features,y)

print('Accuracy is:',clf.score(hog_features,labels)*100,'%')
joblib.dump(clf, "digits_cls.pkl", compress=3)                                                                                             #storing the model into a file called 'digits_cls.pkl'
